# Random Digits Exercice

### Description:
This is an exercise in hypothesis testing, specifically determining if a series of digits [0,9] is random or not, and if the origin is human (i.e. yourself!). In order to determine this, one has to design and apply statistical tests, from which the degree of consistency of being random can be calculated.

### Your task:
Using tests of your choice/design, determine which of the seven data sets are consistent with being random, and which aren't. One is based on the digits you entered in the course questionaire, while the others range from obviously non-random over poor/quasi random to truly random. See if you among these can determine your (human) dataset.

###  Author: 
 - Troels Petersen ([petersen@nbi.dk](mailto:petersen@nbi.dk))

###  Date:
 - 7th of December 2022

---

In [193]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from iminuit import Minuit
from scipy import stats
import os, sys                                         # Modules to see files and folders in directories
from os.path import dirname as parent_folder

In [194]:
sys.path.append('../../External_Functions')
from ExternalFunctions import Chi2Regression, BinnedLH, UnbinnedLH
from ExternalFunctions import nice_string_output, add_text_to_ax    # Useful functions to print fit results on figure

In [195]:
# Set som plotting standards:
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 12}
mpl.rc('font', **font)

---
## Import random numbers from text file:

In [253]:
# Define list of input files:
dataset_letter='A' # between A and G
infiles = ["data_RandomDigits2022_" + dataset_letter + ".txt" ]

In [254]:
# List containing all numbers:
numbers = []

# Loop over input files open them in read mode
for ifile in infiles : 
    with open( ifile, "r" ) as current_file : 
        # Extract current file info : Loop through each line in the file, loop through each character
        # in the line, demand character is not empty ("") and convert the result to an integer
        # Finally add result to the numbers list
        numbers += [int(char) for line in current_file for char in line.strip() if char != ""]

In [260]:
# Always take a look at the data:

numbers = np.array(numbers)
#generate 5000 random numbers
numbers = np.random.randint(0, 10, 5000)

print("#:", len(numbers))
print(numbers[:200])

#: 5000
[3 8 5 0 1 2 2 1 4 8 3 4 2 2 6 8 2 7 7 0 5 5 6 3 2 0 1 7 9 6 5 5 4 2 8 8 6
 2 1 9 1 7 8 0 2 3 8 7 5 3 7 8 0 7 3 4 2 5 2 2 9 6 2 7 9 5 3 8 9 1 6 2 7 5
 7 5 1 9 7 3 7 9 8 2 9 1 4 4 7 9 7 0 8 8 9 8 6 0 6 4 9 2 5 1 3 2 7 3 7 1 8
 4 9 0 5 2 5 8 2 8 9 0 9 7 2 2 0 8 6 6 7 5 8 5 8 0 5 1 8 6 1 1 3 5 7 0 3 4
 4 4 2 9 4 2 8 6 7 9 1 1 5 2 8 5 7 7 6 0 3 4 0 3 6 9 7 1 1 5 5 3 2 3 4 9 2
 3 0 4 6 1 1 5 5 5 7 6 0 3 1 1]


---
## Define your tests:

Here is an example plot from the data, just for convenience. It is all up to you from here...

In [261]:
def is_uniform(data):
    # Find the number of bins that we need
    # For a uniform distribution, we expect each bin to contain approximately the same number of data points
    

    # Calculate the minimum and range of the data
    data_min = np.min(data)
    data_max = np.max(data)
    data_range = data_max - data_min

    num_bins = int(data_range)

    # Create the bins using numpy.histogram
    bins, _ = np.histogram(data, range = (data_min, data_max), bins=num_bins)
    print(bins)
    # Calculate the expected number of data points per bin for a uniform distribution
    expected_bin_size = len(data) / num_bins
    print(expected_bin_size)

    # Calculate percentage of data points in each bin compared to the expected value
    percentage = bins.all() / expected_bin_size
    print(bins.all())

    # Check if the actual number of data points in each bin is within a certain tolerance of the expected value
    tolerance = 0.5  # You can adjust this tolerance as needed
    if np.isclose(bins, expected_bin_size, rtol=tolerance).all():
        return True, 'within tolerance:', tolerance, 'percentage:', percentage
    else:
        return False, 'within tolerance:', tolerance, 'percentage:', percentage


In [262]:
is_uniform(numbers)

[488 460 516 505 505 511 534 505 976]
555.5555555555555
True


(False, 'within tolerance:', 0.5, 'percentage:', 0.0018)

In [263]:
# mean and standard deviation of the data:
mean = np.mean(numbers)
std = np.std(numbers)
print("mean:", mean.round(3), "std:", std.round(3))

print(numbers[:50])

#distance between i and i+1
dist = np.diff(numbers)
print("dist:", dist[:50])


rep_dist = []
run_test = np.zeros(len(numbers))
x_in_row = np.zeros(10)
for i in range(len(numbers)):

    #calculate the number of digits between repetitions
    for j in range(i+1, len(numbers)):
        if numbers[i] == numbers[j] and i != j:
            rep_dist.append(j-i)
            break

    #calculate a runs test
    if i == len(numbers)-1: break
    if numbers[i] < numbers[i+1]:
        run_test[i] = 1
    elif numbers[i] == numbers[i+1]:
        run_test[i] = 0
    else:
        run_test[i] = -1

    #Calculate the number of 2 and 3 repetitions
    if numbers[i] == numbers[i+1] == numbers[i+2] == numbers[i+3] == numbers[i+4] == numbers[i+5] == numbers[i+6] == numbers[i+7] == numbers[i+8] == numbers[i+9]:
        x_in_row[8] += 1
    elif numbers[i] == numbers[i+1] == numbers[i+2] == numbers[i+3] == numbers[i+4] == numbers[i+5] == numbers[i+6] == numbers[i+7] == numbers[i+8]:
        x_in_row[7] += 1
    elif numbers[i] == numbers[i+1] == numbers[i+2] == numbers[i+3] == numbers[i+4] == numbers[i+5] == numbers[i+6] == numbers[i+7]:
        x_in_row[6] += 1
    elif numbers[i] == numbers[i+1] == numbers[i+2] == numbers[i+3] == numbers[i+4] == numbers[i+5] == numbers[i+6]:
        x_in_row[5] += 1
    elif numbers[i] == numbers[i+1] == numbers[i+2] == numbers[i+3] == numbers[i+4] == numbers[i+5]:
        x_in_row[4] += 1
    elif numbers[i] == numbers[i+1] == numbers[i+2] == numbers[i+3] == numbers[i+4]:
        x_in_row[3] += 1
    elif numbers[i] == numbers[i+1] == numbers[i+2] == numbers[i+3]:
        x_in_row[2] += 1
    elif numbers[i] == numbers[i+1] == numbers[i+2]:
        x_in_row[1] += 1
    elif numbers[i] == numbers[i+1]:
        x_in_row[0] += 1

   


        
    
print("rep_dist:", rep_dist[:50])
print("run_test:", run_test[:50])
print("x_in_row:", x_in_row)










mean: 4.529 std: 2.848
[3 8 5 0 1 2 2 1 4 8 3 4 2 2 6 8 2 7 7 0 5 5 6 3 2 0 1 7 9 6 5 5 4 2 8 8 6
 2 1 9 1 7 8 0 2 3 8 7 5 3]
dist: [ 5 -3 -5  1  1  0 -1  3  4 -5  1 -2  0  4  2 -6  5  0 -7  5  0  1 -3 -1
 -2  1  6  2 -3 -1  0 -1 -2  6  0 -2 -4 -1  8 -8  6  1 -8  2  1  5 -1 -2
 -2  4]
rep_dist: [10, 8, 18, 16, 3, 1, 6, 19, 3, 6, 13, 21, 1, 3, 8, 19, 8, 1, 9, 6, 1, 9, 7, 22, 9, 18, 12, 14, 11, 7, 1, 17, 23, 4, 1, 7, 25, 7, 2, 21, 29, 6, 4, 9, 12, 4, 5, 3, 9, 5]
run_test: [ 1. -1. -1.  1.  1.  0. -1.  1.  1. -1.  1. -1.  0.  1.  1. -1.  1.  0.
 -1.  1.  0.  1. -1. -1. -1.  1.  1.  1. -1. -1.  0. -1. -1.  1.  0. -1.
 -1. -1.  1. -1.  1.  1. -1.  1.  1.  1. -1. -1. -1.  1.]
x_in_row: [442.  36.   4.   0.   0.   0.   0.   0.   0.   0.]


In [264]:
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(15, 8))


ax[0,0].hist(numbers, bins=10, range=(-0.5, 9.5), histtype='bar', facecolor = 'gold',ec='k',density=False, lw=2)
ax[0,0].set(xlabel="Numbers", ylabel="Frequency", title="Raw data")

ax[0,1].hist(numbers%2, bins=2, range=(-0.5, 1.5), histtype='bar', facecolor = 'gold',ec='k',density=False, lw=2)         
ax[0,1].set(xlabel="Even (0) or Odd (1)", ylabel="Frequency", title="Even and odd digit test")

ax[0,2].hist(dist, bins=19, range=(-9.5, 9.5), histtype='bar', facecolor = 'gold',ec='k', density=False, lw=2)
ax[0,2].set(xlabel="Distance between two digits", ylabel="Frequency", title="Distance between two digits")

ax[1,0].hist(rep_dist, bins=10, range=(0, 20), histtype='bar', facecolor = 'gold', ec='k',density=False, lw=2)
ax[1,0].set(xlabel="Distance between two repetitions", ylabel="Frequency", title="Distance between two repetitions")

ax[1,1].hist(run_test, bins=3, range=(-1.5, 1.5), histtype='bar', facecolor = 'gold',ec='k',density=False, lw=2)
ax[1,1].set(xlabel="Run test", ylabel="Frequency", title="Test if [i+1] greater than [i]", xticks=([-1,0,1]),xticklabels=['[i]<[i+1]','[i]=[i+1]','[i]>[i+1]'])

ax[1,2].hist(x_in_row, bins=7, range=(-0.5, 6.5), histtype='bar', facecolor = 'gold',ec='k',density=False, lw=2)
ax[1,2].set(xlabel="Number of repetitions", ylabel="Frequency", title="Number of x_in_a_row sequences",xticks=([0,1,2,3,4,5,6]),xticklabels=['2','3','4','5','6','7','8'])







fig.tight_layout()

### Comments on even/odd histogram:

What would you expect this distribution to be? Think about it, and then ask, if the distribution observed in data truly follows your expectation. Afterwards, think about the many other tests, that you can perform.

### Note on general goal of hypothesis tests:
Given the outcome of some test, you should try to convert it to a probability or number of sigmas (same thing). Thus determine what you expect, and then see what you observe and what uncertainty your observation has... and determine if this is consistent with the (null) hypothesis, that the data is truly random digits.

Questions:
---
 1. Discuss with your peers, what criteria truly random numbers should satisfy, and how these criteria can be tested.
    Based on this and on the discussion in class, write up such tests, and run all the datasets through your tests to
    see if some do not seem to be truly random.

 2. While one dataset is truly random, another consists of decimals from $\pi$. Apart from your own data, the other
    data sets are the result of using random numbers, but applying shifting, copying, and using biased selections.
    The non-human data sets are thus supposedly more regular (in some sense) than the humanly generated one. See if
    you can determine which one is "your" (humanly generated) data.

# Learning points:

This exercise should generally make you capable of:
1. Designing (simple) hypothesis tests
2. Coding them up
3. Applying them to data
4. Interpreting the results

You should be highly aware, that a fully correct test may not "discover" anything, even if there is an effect. For example, testing if two samples are alike, and checking if the means are consistent is a test. And they may be consistent, even if the two samples are _very_ different.

However, if just **one** test shows a significant deviation from a hypothesis, then this hypothesis fails.